In [1]:
# Importing necessary libraries
import pandas as pd

In [2]:
file_path = 'Factual Context CF+F+A  - single.csv'

In [3]:
# Reading the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

In [4]:
# # Print the 'Predicted Answer' column
# print("Predicted Answer Column:")
# print(df['Predicted Answer'])

# Print the first row of the 'Predicted Answer' column
print("First row of the 'Predicted Answer' column:")
print(df['Predicted_Answer'].iloc[9])

First row of the 'Predicted Answer' column:
জিয়ান ও সুনেওর


In [5]:
df

,Question,Answer,Predicted_Answer
0,Question: কোন জাদুঘরে সারগনিক বিজয় ফলক আছে?\nC...,parametric answer: ল্যুভর জাদুঘরে\ncontextual ...,ল্যুভর
1,Question: ল্যুভর জাদুঘরে দুটি সারগনিক বিজয় ফল...,parametric answer: খুব সম্ভবত মেসোপটেমিয়া ত্থ...,সুসা
2,Question: ল্যুভর জাদুঘরে দুটি সারগনিক বিজয় ফল...,parametric answer: দ্বাদশ শতাব্দীতে\ncontextua...,দ্বাদশ
3,Question: সরগন কোন ভাষার লিপির প্রসার ঘটিয়ে ছি...,parametric answer: সেমেটিক (আক্কাদীয়ান)\ncont...,সেমেটিক (আক্কাদীয়ান)
4,Question: সুমেরিয়ান কারা?\nContext:<p> আস্যাইর...,parametric answer: nan\ncontextual answer: nan,NaN
...,...,...,...
1315,Question: পানামা উপসাগরে সর্বোচ্চ গভীরতা কত?\n...,parametric answer: ২২০ মিটার\ncontextual answe...,২২০ মিটার
1316,Question: পানামা উপসাগরে কয়টি দ্বীপও আছে?\nCon...,parametric answer: nan\ncontextual answer: nan,NaN
1317,Question: সাংবাদিকতা বলতে কী বুঝায়?\nContext:<...,"parametric answer: বিভিন্ন ঘটনাবলী, বিষয়, ধার...","বিভিন্ন ঘটনাবলী, বিষয়, ধারণা, মানুষ, প্রকৃতি,..."
1318,Question: কোন দশকে আধুনিক সাংবাদিকতা রূপ ধারণ ...,parametric answer: ১৯২০\ncontextual answer: ১৯২০,১৯২০


In [ ]:
import google.generativeai as genai

genai.configure(api_key="")
model = genai.GenerativeModel("gemini-2.0-flash-exp")

response = model.generate_content('''
Target:
parametric answer: সাতোশি টোবা
contextual answer: সাতোশি টোবা


Generated:
Contextual Answer: সাতোশি তোবা
Parametric Answer: শিনিচিরো টোবা



In the above section compare bewtween traget and contextual and both of them are in Bengali. You need to just compare the answers, you will check if they are presenting same meaning, do not check length if they are meaning the same asnwer or not. If the generated answer is long but giving the actual value like target provide me good result and provide me score in following format in %

Format:
Contextual Answer smilirarity:
Parametric Answer similarity:
''')
print(response.text)

Contextual Answer similarity: 100%
Parametric Answer similarity: 0%



In [ ]:
import pandas as pd
import google.generativeai as genai
from tqdm import tqdm
import time

# Configure the Gemini API
genai.configure(api_key="")
model = genai.GenerativeModel("gemini-2.0-flash-exp")

# Function to calculate similarity using Gemini API
def calculate_similarity(row):
    """
    Compare target and generated answers and return the similarity scores.
    """
    # Skip rows with NaN, "হ্যাঁ", "না", or specific conditions 
    answer = str(row['Answer']) if not pd.isna(row['Answer']) else "" 
    predicted_answer = str(row['Predicted_Answer']) if not pd.isna(row['Predicted_Answer']) else "" 
     
    if any(keyword in answer for keyword in ["হ্যাঁ", "nan"]) or "does not provide" in predicted_answer: 
        return None
        
    if any(keyword in answer for keyword in ["হ্যাঁ",  "nan", "None"]) or "does not provide" in row['Answer']: 
        return None

    target = row['Answer']
    generated = row['Predicted_Answer']
    #question = row['Question']
    # try:
    #     paraanswer = generated.split("Parametric Answer: ")[1].strip()
    #     contextanswer = generated.split("Parametric Answer: ")[0].lstrip("Contextual Answer:").strip()
    # except:
    #     return None

    # final_generated = f"parametric answer:{paraanswer}\ncontextual answer: {contextanswer}"
    print("Target:",target)
    print("generated:",generated)

    # Check for None, NaN, or empty values
    if pd.isna(target) or pd.isna(generated) or not target.strip() or not generated.strip():
        return {"Contextual": "", "Parametric": ""}

    try:
        # Prepare the prompt for Gemini
        prompt = f'''
Target:
{target}

Generated:
parametric answer: {generated}, contextual answer: {generated}



In the above section compare bewtween traget and contextual and both of them are in Bengali. You need to just compare the answers, you will check if they are presenting same meaning, do not check length if they are meaning the same asnwer or not. If the generated answer is long but giving the actual value like target provide me good result and provide me score in following format in %
Also do not provide 0% directly if the value is close give percentage according to that.

Format:
Contextual Answer similarity: [Value in %]
Parametric Answer similarity: [Value in %]
'''
        # Generate response
        response = model.generate_content(prompt)
        result = response.text

        # Extract similarity scores from the response
        contextual_similarity = ""
        parametric_similarity = ""
        for line in result.splitlines():
            if "Contextual Answer similarity:" in line:
                contextual_similarity = line.split(":")[-1].strip()
            elif "Parametric Answer similarity:" in line:
                parametric_similarity = line.split(":")[-1].strip()

        return {
            "Contextual": contextual_similarity,
            "Parametric": parametric_similarity
        }
    except Exception as e:
        print(f"Error processing row: {e}")
        return {"Contextual": "", "Parametric": ""}

# Load the DataFrame
# Assuming df is already loaded before this point

# Add new columns for similarity scores
df['Contextual Similarity'] = ""
df['Parametric Similarity'] = ""

# Process each row and calculate similarity with progress bar
for idx in tqdm(range(0, 700), desc="Processing rows", unit="row"):
    row = df.iloc[idx]
    
    # Check if row should be skipped
    similarity_scores = calculate_similarity(row)
    
    # Skip this iteration if similarity_scores is None
    if similarity_scores is None:
        continue
    
    # Update the DataFrame with similarity scores
    df.at[idx, 'Contextual Similarity'] = similarity_scores['Contextual']
    df.at[idx, 'Parametric Similarity'] = similarity_scores['Parametric']

    # Save after every row to avoid data loss
    df.to_csv('processed_results_factual_T5_multiple.csv', index=False)

    # Rate limiting to comply with API usage policies
    time.sleep(13)  # Adjust the time if needed

print("Processing completed. Results saved to 'processed_results.csv'")

Processing rows:   0%|                                                                        | 0/700 [00:00<?, ?row/s]

Target: parametric answer: ল্যুভর জাদুঘরে
contextual answer: ল্যুভর জাদুঘরে
generated: ল্যুভর


Processing rows:   0%|                                                              | 1/700 [00:17<3:21:59, 17.34s/row]

Target: parametric answer: খুব সম্ভবত মেসোপটেমিয়া ত্থেকে
contextual answer: খুব সম্ভবত মেসোপটেমিয়া ত্থেকে
generated: সুসা


Processing rows:   0%|▏                                                             | 2/700 [00:32<3:13:31, 16.63s/row]

...

Target: parametric answer: এমন একটি পরিষেবা যা নার্সিং মায়েদের দ্বারা দান করা মানব দুধ সংগ্রহ, প্রক্রিয়াকরন এবং সংরক্ষনের পর তা সরবরাহকারীর সাথে জৈবিকভাবে সম্পর্কিত নয় এমন নবজাতক শিশুকে সরবরাহ করে
contextual answer: এমন একটি পরিষেবা যা নার্সিং মায়েদের দ্বারা দান করা মানব দুধ সংগ্রহ, প্রক্রিয়াকরন এবং সংরক্ষনের পর তা সরবরাহকারীর সাথে জৈবিকভাবে সম্পর্কিত নয় এমন নবজাতক শিশুকে সরবরাহ করে
generated: নার্স মায়েদের দ্বারা দান করা মানব দুধ সংগ্রহ, প্রক্রিয়াকরন এবং সংরক্ষনের পর তা সরবরাহকারীর সাথে জৈবিকভাবে সম্পর্কিত নয় এমন নবজাতক শিশুকে সরবরাহ করা হয়


Processing rows: 100%|███████████████████████████████████████████████████████████▉| 699/700 [1:57:23<00:09,  9.94s/row]

Target: parametric answer: প্রথম বছর স্তন্যপান করা
contextual answer: প্রথম বছর স্তন্যপান করা
generated: প্রথম বছর স্তন্যপান করা


Processing rows: 100%|████████████████████████████████████████████████████████████| 700/700 [1:57:42<00:00, 12.73s/row]


Processing completed. Results saved to 'processed_results.csv'
